# Asynchronous batching

This notebook shows how to use Asynchronous Batching to halve the cost of LM calls.

Implementation relies on LiteLLM's own batch sdk ([link to docs](https://docs.litellm.ai/docs/batches)).

In [1]:
import dspy
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
gpt_4_1 = dspy.LM("openai/gpt-4.1", api_key=api_key)
dspy.configure(lm=gpt_4_1)

In [3]:
questions = ["1+1?", "2+2?", "3+3?"]
examples = [
    dspy.Example(
        question=q
    ).with_inputs("question")
    for q in questions
]
examples

[Example({'question': '1+1?'}) (input_keys={'question'}),
 Example({'question': '2+2?'}) (input_keys={'question'}),
 Example({'question': '3+3?'}) (input_keys={'question'})]

In [4]:
mathbot = dspy.Predict("question -> answer")

## First method (fine-grained steps)

The steps are:
1. create a jsonl file (`module.create_batch_file()`) 
2. send it to the endpoint (`module.asubmit_batch_file()`) 
3. optionally listen for its completion (`module.aretrieve_batch()`) 
4. and finally retrieve the predictions (`mathbot.aretrieve_batch_predictions()`) 

In [7]:
# 1. create a jsonl file
artifacts = mathbot.create_batch_file(
    examples,
    input_file_path="batches/math_input.jsonl",
)
artifacts

BatchRequestArtifacts(request_file=WindowsPath('batches/math_input.jsonl'), metadata_file=WindowsPath('batches/math_input.jsonl.metadata.json'), metadata={'generated_at': '2025-11-29T09:51:45.915000Z', 'request_file': 'batches\\math_input.jsonl', 'endpoint': '/v1/chat/completions', 'dependency_versions': {'python': '3.13', 'dspy': '3.1.0b1', 'cloudpickle': '3.1'}, 'module': {'class_path': 'dspy.predict.predict.Predict', 'repr': "Predict(StringSignature(question -> answer\n    instructions='Given the fields `question`, produce the fields `answer`.'\n    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})\n    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})\n))"}, 'lm': {'model': 'gpt-4.1', 'provider': 'openai', 'type': 'chat'}, 'examples': [{'custom_id': 'predict-0-a018f472', 'index': 0, 'inputs': {'question': 

In [8]:
# 2. send it to the endpoint
batch_handle = await mathbot.asubmit_batch_file(
    artifacts,
    completion_window="24h",
)

In [9]:
# 3. optionally listen for its completion
import asyncio

while True:
    info = await mathbot.aretrieve_batch(batch_handle.batch_id)
    if getattr(info, "status", None) == "completed" and getattr(info, "output_file_id", None):
        break
    await asyncio.sleep(5)

In [10]:
# 4. and finally retrieve the predictions
predictions = await mathbot.aretrieve_batch_predictions(
    batch_handle.batch_id,
    artifacts,
    download_output_path="batches/math_outputs.jsonl",  # optional, defaults next to metadata
)

In [11]:
predictions

[Prediction(
     answer='2'
 ),
 Prediction(
     answer='2+2 equals 4.'
 ),
 Prediction(
     answer='3+3 equals 6.'
 )]

## Second method (the lazy one-liner)

`module.abatch()` is a wrapper around the previous steps. It doesn't create files on disk.

In [5]:
predictions2 = await mathbot.abatch(
    examples=examples
)

In [6]:
predictions2

[Prediction(
     answer='1+1 = 2'
 ),
 Prediction(
     answer='2+2 equals 4.'
 ),
 Prediction(
     answer='3+3=6'
 )]